In [ ]:
import os
import torch
from torch.cuda import device
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# reading all images from directory
Images = {}
Folder = 'Grouped_Output'
for filename in os.listdir(Folder):
    # opening all folders in this directory and saving the images in a dictionary
    Images[filename] = []
    for img in os.listdir(Folder + '/' + filename):
        Images[filename].append(img)

In [ ]:
# # printing one image from each folder
# for folder in Images.keys():
#     img = Image.open(Folder + '/' + folder + '/' + Images[folder][0])
#     plt.imshow(np.array(img))
#     plt.show()

In [ ]:
# Pre-processing the images 
# 1. Resizing the images to 64x64
# 2. Normalizing the images
transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor()])

In [ ]:
# applying transformation to all images
Images_tensor = {}
for folder in Images.keys():
    Images_tensor[folder] = []
    for img in Images[folder]:
        img = Image.open(Folder + '/' + folder + '/' + img)
        img = transform(img)
        Images_tensor[folder].append(img)

## VAE Implementation:  
1. Define the encoder and decoder networks.
2. Define the VAE loss function (reconstruction loss + KL divergence).
3. Train the VAE on the augmented dataset

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Encoder(nn.Module):
    def __init__(self, latent_dim):
        super(Encoder, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1)
        self.fc_mu = nn.Linear(256 * 4 * 4, latent_dim)
        self.fc_logvar = nn.Linear(256 * 4 * 4, latent_dim)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = x.view(x.size(0), -1)
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar

In [ ]:
class Decoder(nn.Module):
    def __init__(self, latent_dim):
        super(Decoder, self).__init__()
        self.fc = nn.Linear(latent_dim, 256 * 4 * 4)
        self.deconv1 = nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1)
        self.deconv2 = nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1)
        self.deconv3 = nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1)
        self.deconv4 = nn.ConvTranspose2d(32, 3, kernel_size=4, stride=2, padding=1)

    def forward(self, z):
        z = self.fc(z)
        z = z.view(z.size(0), 256, 4, 4)
        z = F.relu(self.deconv1(z))
        z = F.relu(self.deconv2(z))
        z = F.relu(self.deconv3(z))
        z = torch.sigmoid(self.deconv4(z))
        return z

In [ ]:
class VAE(nn.Module):
    def __init__(self, latent_dim):
        super(VAE, self).__init__()
        self.encoder = Encoder(latent_dim)
        self.decoder = Decoder(latent_dim)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = self.reparameterize(mu, logvar)
        return self.decoder(z), mu, logvar

In [ ]:
def vae_loss(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [ ]:
# Training loop
vae = VAE(latent_dim=20)
optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)

# Function to check tensor dimensions
def print_tensor_info(tensor, name):
    print(f"{name} shape: {tensor.shape}")
    print(f"{name} size: {tensor.numel()}")
    print(f"{name} dtype: {tensor.dtype}")
    print(f"{name} device: {tensor.device}")
    print(f"{name} requires_grad: {tensor.requires_grad}")
    print("---")

# Preprocess images
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

In [ ]:
# Apply transformation to all images
Images_tensor = {}
for folder in Images.keys():
    Images_tensor[folder] = []
    for img_path in Images[folder]:
        img = Image.open(os.path.join(Folder, folder, img_path))
        img = transform(img)
        Images_tensor[folder].append(img)

In [ ]:
epoch_Number = 10
for epoch in range(epoch_Number):
    for folder in Images_tensor.keys():
        for img in Images_tensor[folder]:
            img = img.unsqueeze(0)  # Add batch dimension
            print_tensor_info(img, "Input image")  # Debug info
            recon_img, mu, logvar = vae(img)
            loss = vae_loss(recon_img, img, mu, logvar)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    print(f'Epoch: {epoch}, Loss: {loss.item()}')

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.fc1 = nn.Linear(100, 256)
        self.fc2 = nn.Linear(256, 512)
        self.fc3 = nn.Linear(512, 1024)
        self.fc4 = nn.Linear(1024, 64*64)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = torch.tanh(self.fc4(x))
        return x

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(64*64, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 1)

    def forward(self, x):
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = torch.sigmoid(self.fc4(x))
        return x

# Initialize models
generator = Generator()
discriminator = Discriminator()

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.0002)
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0002)

# Loss function
adversarial_loss = nn.BCELoss()

In [ ]:
# Training
epoch_Number = 10
for epoch in range(epoch_Number):
    for folder in Images_tensor.keys():
        for img in Images_tensor[folder]:
            img = img.view(-1, 64*64)
            z = torch.randn(1, 100)
            fake_img = generator(z)
            real_label = torch.ones(1, 1)
            fake_label = torch.zeros(1, 1)

            # Train Discriminator
            optimizer_D.zero_grad()
            real_output = discriminator(img)
            fake_output = discriminator(fake_img)
            real_loss = adversarial_loss(real_output, real_label)
            fake_loss = adversarial_loss(fake_output, fake_label)
            d_loss = real_loss + fake_loss
            d_loss.backward()
            optimizer_D.step()

            # Train Generator
            optimizer_G.zero_grad()
            z = torch.randn(1, 100)
            fake_img = generator(z)
            output = discriminator(fake_img)
            g_loss = adversarial_loss(output, real_label)
            g_loss.backward()
            optimizer_G.step()
    print(f'Epoch: {epoch}, D Loss: {d_loss.item()}, G Loss: {g_loss.item()}')

In [ ]:
# Generating images
z = torch.randn(1, 100)
fake_img = generator(z)
fake_img = fake_img.view(64, 64)
plt.imshow(fake_img.detach().numpy())
plt.show()

In [ ]:
# saving the model
torch.save(generator.state_dict(), 'generator.pth')
torch.save(discriminator.state_dict(), 'discriminator.pth')

In [ ]:
# genreating image from group 7
z = torch.randn(1, 100)
fake_img = generator(z)
fake_img = fake_img.view(64, 64)
# saving the image
plt.imshow(fake_img.detach().numpy())
plt.savefig('Generated_Image.png')